In [1]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple
from scipy.spatial.distance import cosine as cosine_distance
from collections import defaultdict


In [2]:
with open("queries2.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]


In [3]:
for i,q in enumerate(queries):
    try:
        assert len(q.split("\t")) == 2
    except:
        print(i, q, len(q.split("\t")))

In [4]:
for i,q in enumerate(queries):
    a = q.count("arg1:") == q.count("arg2:") == 1
    if not a:
        print(i,q)

In [5]:
queries2results = defaultdict(list)
id2query = defaultdict(list)

for id_and_query in queries:
    query,id = id_and_query.split("\t")
    id2query[int(id)].append(query)
    
print(id2query[1])

['<>arg1:virus $infection $causes a <>arg2:condition', '<>arg1:virus $infection, which $causes a <>arg2:condition', '<>arg1:virus $infection $results $in <>arg2:condition', '<>arg1:virus $infection, which $results $in <>arg2:condition', '<>arg2:condition $is $the $result of <>arg1:virus $infection', '<>arg2:condition, which $is $the $result of <>arg1:virus $infection', '<>arg2:condition $is $[l=cause|trigger]caused $by <>arg1:virus $infection', 'as a result of <>arg1:virus $infection, <>arg2:condition $can $develop']


In [6]:
# dataset_name = "covid19"
# num_results = 100
# query_type = "syntactic"

# for i, q_and_id in tqdm.tqdm(enumerate(queries), total = len(queries)):
#     q, id = q_and_id.split("\t")
#     id = int(id)
#     try:
#         df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
#         queries2results[id].append(df)
#         time.sleep(2)
#     except Exception as e:
#         print("Error", i+1)
#         print(e)
        
#         continue

In [7]:
# for id, dfs in queries2results.items():
#      for i,df in enumerate(dfs):
#          print(id, i)
#          queries2results[id][i] = df.dropna(subset=['arg1_first_index',"arg1_last_index","arg2_first_index","arg2_last_index"])

In [11]:
for id, dfs in queries2results.items():
      for i,df in enumerate(dfs):
            queries2results[id][i]["spike_query"] = id2query[id][i]

In [9]:
# with open("queries2results.pickle", "wb") as f:
#       pickle.dump(queries2results, f)

In [10]:
with open("queries2results.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [12]:
queries2results.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56])

In [13]:
# queries2results_new = dict()
# for k,dfs in queries2results.items():
#     if k == 0: continue
#     queries2results_new[k] = pd.concat(dfs, axis=0)
    
# queries2results = queries2results_new

In [39]:
ids = set([int(l.split("\t")[-1]) for l in queries])

In [40]:
def add_arguments(sent:str, arg1_start, arg1_end, arg2_start, arg2_end):
    
        s_lst = sent.split(" ")
        if arg1_start > arg2_start:
            arg1_start, arg2_start = arg2_start, arg1_start
            arg1_end, arg2_end = arg2_end, arg1_end
            arg1_str, arg2_str = "<<ARG2:", "<<ARG1:"
        else:
            arg1_str, arg2_str = "<<ARG1:", "<<ARG2:"
        
        s_with_args = s_lst[:arg1_start] + [arg1_str] + s_lst[arg1_start:arg1_end+1] + [">>"] + s_lst[arg1_end+1:arg2_start] + [arg2_str] + s_lst[arg2_start:arg2_end+1] + [">>"] +s_lst[arg2_end+1:]  
        #s_with_args = s_lst[:arg1_start] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args).replace("ARG1: ", "ARG1:").replace("ARG2: ", "ARG2:")
        s_with_args = s_with_args.replace(" >>", ">>")
        return s_with_args
    

In [41]:
NUM_PAIRS_PER_RELATION = 500

data = []

for id in tqdm.tqdm(ids): #foreach relation
    
    pattern_pairs = list(itertools.product(range(len(id2query[id])), repeat=2))
    pattern_pairs_different = [(p1,p2) for (p1,p2) in pattern_pairs if p1 != p2]
    
    sum_pattern = 0
    relation_examples = []
    
    for p1, p2 in pattern_pairs_different:
        df1,df2 = queries2results[id][p1], queries2results[id][p2]
        if df1.empty or df2.empty: continue
            
        sentences1, sentences2 = df1["sentence_text"].tolist(), df2["sentence_text"].tolist()
        query1, query2 = df1["spike_query"].tolist()[0], df2["spike_query"].tolist()[0]
        arg1_first_start, arg1_first_end = df1["arg1_first_index"].tolist(), df1["arg1_last_index"].tolist()
        arg2_first_start, arg2_first_end = df1["arg2_first_index"].tolist(), df1["arg2_last_index"].tolist()
        arg1_second_start, arg1_second_end = df2["arg1_first_index"].tolist(), df2["arg1_last_index"].tolist()
        arg2_second_start, arg2_second_end = df2["arg2_first_index"].tolist(), df2["arg2_last_index"].tolist()
        
        
        all_pair_combinations = list(itertools.product(range(len(df1)), range(len(df2))))
        random.shuffle(all_pair_combinations)
        for combination in all_pair_combinations[:100]:
            ind1, ind2 = combination
            
            sent1, sent2 = sentences1[ind1], sentences2[ind2]
            sent1_arg1_start, sent1_arg1_end = arg1_first_start[ind1], arg1_first_end[ind1]
            sent1_arg2_start, sent1_arg2_end = arg2_first_start[ind1], arg2_first_end[ind1]
            sent2_arg1_start, sent2_arg1_end = arg1_second_start[ind2], arg1_second_end[ind2]
            sent2_arg2_start, sent2_arg2_end = arg2_second_start[ind2], arg2_second_end[ind2]
            sent1_with_args = add_arguments(sent1, sent1_arg1_start, sent1_arg1_end, sent1_arg2_start, sent1_arg2_end)
            sent2_with_args = add_arguments(sent2, sent2_arg1_start, sent2_arg1_end, sent2_arg2_start, sent2_arg2_end)
            
            sent1_lst = sent1.split(" ")
            sent2_lst = sent2.split(" ")
            
            sent1_arg1_w = sent1_lst[sent1_arg1_start:sent1_arg1_end+1]
            sent1_arg2_w = sent1_lst[sent1_arg2_start:sent1_arg2_end+1]
            sent2_arg1_w = sent2_lst[sent2_arg1_start:sent2_arg1_end+1]
            sent2_arg2_w = sent2_lst[sent2_arg2_start:sent2_arg2_end+1]
            
            d = {"first": sent1_with_args, "second": sent2, "second_with_arguments": sent2_with_args, "query_first": query1, "query_second": query2, 
                 "first_arg1": (sent1_arg1_start, sent1_arg1_end), "first_arg2": (sent1_arg2_start, sent1_arg2_end), 
                 "second_arg1": (sent2_arg1_start, sent2_arg1_end),
                 "second_arg2": (sent2_arg2_start, sent2_arg2_end), "first_arg1_words": sent1_arg2_w,
                "first_arg2_words": sent1_arg2_w, "second_arg1_words": sent2_arg1_w, "second_arg2_words": sent2_arg2_w,
                "query_id": id}
            relation_examples.append(d)
    
    random.shuffle(relation_examples)
    relation_examples = relation_examples[:NUM_PAIRS_PER_RELATION]
    data.extend(relation_examples)
    
random.shuffle(data)

100%|██████████| 56/56 [00:09<00:00,  5.75it/s]


In [49]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]
data = [d for i,d in enumerate(data) if lengths[i] < 250]
data = [d for d in data if (d["second_arg1"] != d["second_arg2"]) and (d["first_arg1"] != d["first_arg2"])]
data = [d for d in data if d["first_arg1_words"] != [''] and d["first_arg2_words"] != ['']
       and d["second_arg1_words"] != [''] and d["second_arg2_words"] != ['']]
ids_subset = np.random.choice(list(ids), size = int(0.2 * len(ids)))
data_train = [d for d in data if d["query_id"] not in ids_subset]
data_dev = [d for d in data if d["query_id"] in ids_subset]
print(ids_subset)

[10 23 22 12 36 33 54 11 22 13 11]


In [48]:
with open("data.pickle", "wb") as f:
    pickle.dump(data, f)
    
with open("data_dev.pickle", "wb") as f:
    pickle.dump(data_dev, f)
    
with open("data_train.pickle", "wb") as f:
    pickle.dump(data_train, f)
    
import json

def write(data, fname):
    
    with open(fname, "w") as f:
        for d in data:
            first, second = d["first"], d["second_with_arguments"]
            first_arg1 = d["first_arg1"]
            first_arg2 = d["first_arg2"]
            second_arg1 = d["second_arg1"]
            second_arg2 = d["second_arg2"]
        
            elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
            keys = ["first", "second", "first_arg1", "first_arg2"]
        
            f.write(json.dumps(d) + "\n")
        
            #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")
            
    
write(data, "data.txt")
write(data_dev, "data_dev.txt")
write(data_train, "data_train.txt")

In [47]:
len(data_train), len(data_dev)

(19738, 4708)

In [29]:
for i in range(20):
    print(data[i]["first"], data[i]["first_arg1"], data[i]["first_arg2"])
    print(data[i]["second_with_arguments"])
    print("---------------")
    print(data[i]["query_first"])
    print(data[i]["query_second"])
    print("============================")

The structure of coronavirus 3C-like proteinase contains three domains , the first two domains form a chymotrypsin fold , which is responsible for the catalytic reaction , and <<ARG1:the extra helical domain of the enzyme>> plays an important role in controlling the associationdissociation equilibrium and regulating <<ARG2:the activity and specificity of the enzyme>> [ 6 ] . (28, 34) (46, 52)
In rotavirus infected cells an increased Akt phosphorylation depends on a direct interaction between <<ARG1:the viral nonstructural protein NSP1>> and <<ARG2:PI3 K>> .
---------------
the $enzyme <>arg1:A $[l]regulates the $enzyme <>arg2:B
the $[l]interaction between <>arg1:[e=PROTEIN]A and <>arg2:[e=PROTEIN]B
Specimens 27 and 199 reacted strongly with all Ad41 DNA probes , although types Ad2 and <<ARG1:Ad5>> emerged from <<ARG2:culture>> . (16, 16) (19, 19)
Kohl et al. [ 25 ] carried out genome pyrosequencing of bat adenovirus 2 and found there was an interspecies transmission during evolution of

In [30]:
print(len(data))

24437


In [31]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]

In [32]:
np.mean(lengths), np.std(lengths), np.max(lengths), np.min(lengths)

(67.92299198429576, 22.057857454680867, 237, 16)

In [58]:
len([l for l in lengths if l > 250])

0